In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import START, END, StateGraph
from typing import List, TypedDict
from CONFIG import OPENAI_EMBEDDED_MODEL, GROQ_MODEL
from langchain_community.vectorstores import FAISS
import re
from pydantic import BaseModel
from langchain_core.documents import Document

In [18]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL)
embedded_model = OpenAIEmbeddings(model=OPENAI_EMBEDDED_MODEL)

In [19]:
pdf_loading = PyPDFLoader(file_path='A:\AI_Projects_Practice\CRAG\The_Evolution_of_AI_in_Dubai.pdf')
pdf = pdf_loading.load()
len(pdf)

8

In [20]:
splitting = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100).split_documents(pdf)
len(splitting)

56

In [21]:
vector_storage = FAISS.from_documents(splitting, embedded_model)

In [22]:
retrieved = vector_storage.as_retriever(search_type='similarity', search_kwargs={'k': 2})
for i in retrieved.invoke('dubai'):
    print(i.page_content)
    print('*'*100)

boosting the economy, and improving quality of life for residents and visitors.
Key components of this strategy include:
Infrastructural Development: Dubai is investing in tech infrastructure to support AI-driven
services. This entails deploying high-speed internet and cloud computing resources across the
city.
****************************************************************************************************
Dubai AI Strategy: Introduced in 2017, this strategy aims to make Dubai a global hub for AI by
2031. It focuses on utilizing AI to optimize government operations, improve city services, and
advance economic development. It involves collaboration between governmental bodies and
private sectors to create an ecosystem that fosters AI innovation.
****************************************************************************************************


In [23]:
UPPER_THD = 0.7
LOWER_THD = 0.3

In [24]:
class state(TypedDict):
    question: str
    doc: List[Document]

    good_docs: List[Document]
    verdict: str
    reason: str

    strip: List[str]
    refined_strip: List[str]
    refined_text: str
    
    answer: str

In [25]:
def retrieved_node(state):
    return {'doc': retrieved.invoke(state['question'])}

In [33]:
# EVALUATOR

class DocEvaluator(BaseModel):
    score: float
    reason: str

doc_eval_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a strict retrieval evaluator for RAG.\n"
            "You will be given ONE retrieved chunk and a question.\n"
            "Return a relevance score in [0.0, 1.0].\n"
            "- 1.0: chunk alone is sufficient to answer fully/mostly\n"
            "- 0.0: chunk is irrelevant\n"
            "Be conservative with high scores.\n"
            "Also return a short reason.\n"
            "Output JSON only."
	  ),
	  (
		"human",
		"Question is: {question}\n\nChunk is: {chunk}"
	  )
    ]
)

doc_eval_chain = doc_eval_prompt | llm.with_structured_output(DocEvaluator)

def eval_each_doc_node(state: state) -> state:

    q = state["question"]
    
    scores: List[float] = []
    reasons: List[str] = []
    good: List[Document] = []

    for d in state["docs"]:
        out = doc_eval_chain.invoke({"question": q, "chunk": d.page_content})
        scores.append(out.score)
        reasons.append(out.reason)

        # 5) for CORRECT case we will refine only docs with score > LOWER_TH
        if out.score > LOWER_THD:
            good.append(d)

    # 2) CORRECT if at least one doc > UPPER_TH
    if any(s > UPPER_THD for s in scores):
        return {
            "good_docs": good,
            "verdict": "CORRECT",
            "reason": f"At least one retrieved chunk scored > {UPPER_THD}.",
        }
    
        # 3) INCORRECT if all docs < LOWER_TH
    if len(scores) > 0 and all(s < LOWER_THD for s in scores):
        why = "No chunk was sufficient."
        return {
            "good_docs": [],
            "verdict": "INCORRECT",
            "reason": f"All retrieved chunks scored < {LOWER_THD}. {why}",
        }

    # 4) Anything in between => AMBIGUOUS
    why = "Mixed relevance signals."
    return {
        "good_docs": good,
        "verdict": "AMBIGUOUS",
        "reason": f"No chunk scored > {UPPER_THD}, but not all were < {LOWER_THD}. {why}",
    }

In [35]:
def decompose_to_sentence(text: str) -> List[str]:
    text = re.sub(r"\s+", " ", text).strip()
    sentence = re.split(r"(?<=[.!?])\s+", text)
    return [s.strip() for s in sentence if len(s.strip()) > 20]

In [37]:
class KeepOrDrop(BaseModel):
    keep: bool

filter_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a strict relevence filter.\n"
            "Return BOOLEAN, keep=true only if the sentence directly helps answer the question,\n"
            "Use only the sentence, output in JSON format only."
	  ),
	  (
		"human",
		"Question: {question}\n\n"
		"Sentence: {sentence}\n"
	  ),
    ]
)

filter_chain = filter_prompt | llm.with_structured_output(KeepOrDrop)

In [ ]:
def refined(state: state):
    q = state['question']

    # Conbined the text
    context = "\n\n".join(d.page_content for d in state['good_docs']).strip()

    # DECOMPOSITION
    striped = decompose_to_sentence(context)

    # FILTERED
    kept: List[str] = []
    for s in striped:
        if filter_chain.invoke({'question': q, 'sentence': s}).keep:
            kept.append(s)
            
    # RECOMPOSITION
    refined_context = '\n'.join(kept).strip()

    return {
        'strip': striped,
    	  'refined_strip': kept,
        'refined_text': refined_context
    }

In [39]:
prompt = ChatPromptTemplate(
    [
        ("system", "Answer only from the context. If not in context, say you don't know."),
        ("human", "Question is: {question}\n\nRefined Context is: {context}")
    ]
)

def generate_node(state):
    question = state['question']
    context = state['refined_text']
    out = (prompt|llm).invoke(
        {
            "question": question,
            "context": context
	  }
    )

    return {'answer':out.content}

In [40]:
def fail_node(state: state) -> state:
    return {"answer": f"FAIL: {state['reason']}"}

def ambiguous_node(state: state) -> state:
    return {"answer": f"Ambiguous: {state['reason']}"}

def route_after_eval(state: state) -> str:
    if state["verdict"] == "CORRECT":
        return "refine"
    elif state["verdict"] == "INCORRECT":
        return "web_search"
    else:
        return "ambiguous"

In [ ]:
g = StateGraph(state)
g.add_node("retrieve", retrieved_node)
g.add_node("eval_each_doc", eval_each_doc_node)
g.add_node("refine", refined)
g.add_node("generate", generate_node)
g.add_node("fail", fail_node)
g.add_node("ambiguous", ambiguous_node)


g.add_edge(START, "retrieve")
g.add_edge("retrieve", "eval_each_doc")

g.add_conditional_edges(
    "eval_each_doc",
    route_after_eval,
    {"refine": "refine", "web_search": "fail", "ambiguous": "ambiguous"}
)
g.add_edge("refine", "generate")
g.add_edge("generate", END)
g.add_edge("fail", END)

app = g.compile()
app